<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/China_A_share_market_tushare.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [1]:
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'condacolab'

Install other libraries

In [ ]:
!pip install stockstats
!pip install tushare
#install talib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib') 
!./configure --prefix=/usr
!make
!make install
#!sudo make install # Sometimes it need root 
os.chdir('../')
!pip install TA-Lib

In [20]:
%cd /
!git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
%cd /FinRL-Meta/

/
fatal: destination path 'FinRL-Meta' already exists and is not an empty directory.
/FinRL-Meta


In [1]:
import os 
os.getcwd()

'/Users/hchen/Desktop/FinRL-Tutorials/1-Introduction'

In [2]:
os.environ["meta"]='/home/youyou/workspace/FinRL-Tutorials'

## Import Modules

In [3]:
import sys
sys.path.append('/Users/hchen/Desktop/FinRL-Tutorials')
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
from IPython import display

display.set_matplotlib_formats("svg")

from meta import config 
from meta.data_processor import DataProcessor 
from main import check_and_make_directories 
from meta.data_processors.tushare import Tushare, ReturnPlotter 
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv 
from agents.stablebaselines3_models import DRLAgent 
import os 
from typing import List 
from argparse import ArgumentParser 
from meta import config 
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


## Create Folders

In [4]:
import os

''' 
use check_and_make_directories() to replace the following
conda
if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''


check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

## Download data, cleaning and feature engineering

In [5]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

#ticker_list = [ '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2020-01-03'

TIME_INTERVAL = "1d" 
kwargs = {} 
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5' 
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

tushare successfully connected


### Download and Clean

In [6]:
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

# p.load_data("./data/dataset.csv")


 40%|████      | 6/15 [00:45<01:37, 10.86s/it]

HTTPConnectionPool(host='127.0.0.1', port=15236): Read timed out. (read timeout=30)


 60%|██████    | 9/15 [01:34<01:18, 13.10s/it]

HTTPConnectionPool(host='127.0.0.1', port=15236): Read timed out. (read timeout=30)
HTTPConnectionPool(host='127.0.0.1', port=15236): Read timed out. (read timeout=30)


100%|██████████| 15/15 [03:20<00:00, 13.40s/it]


Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (17960, 8)
Shape of DataFrame:  (18315, 8)


In [7]:
p.dataframe.head()

,tic,time,open,high,low,close,adjusted_close,volume
0,600000.SH,2015-01-05,15.88,16.25,15.56,16.07,16.07,5135687.09
1,600009.SH,2015-01-05,19.82,20.91,19.82,20.53,20.53,371485.54
2,600016.SH,2015-01-05,10.87,10.96,10.50,10.78,10.78,9138873.70
3,600028.SH,2015-01-05,6.59,7.14,6.45,7.14,7.14,11864996.45
4,600030.SH,2015-01-05,33.90,35.25,33.01,34.66,34.66,6986272.15


### Add technical indicator

In [8]:
p.add_technical_indicator(config.INDICATORS) 
p.fillna()

p.dataframe.head()

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
1,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
2,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
3,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
4,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


## Split training dataset

In [9]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [10]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']


In [11]:
train.head()

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


In [12]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [13]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


In [14]:
train.head()

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


## Train

In [15]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }
# train
e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [16]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


### DDPG

In [17]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg/ddpg_13


Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 976438.59
total_reward: -23561.41
total_cost: 12149.56
total_trades: 16679
Sharpe: 0.147
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 996647.70
total_reward: -3352.30
total_cost: 121.30
total_trades: 16680
Sharpe: 0.374
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 1007973.81
total_reward: 7973.81
total_cost: 102.19
total_trades: 16680
Sharpe: 0.299
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 999897.82
total_reward: -102.18
total_cost: 102.18
total_trades: 16680
Sharpe: 0.365
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 81        |
|    time_elapsed    | 54        |
|    total_timesteps | 4452      |
| train/             |           |
|    actor_loss      | 49.3      |
|    critic_loss     | 139       |
|    learning_rate 

### A2C

In [18]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_9


------------------------------------
| time/                 |          |
|    fps                | 360      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -2.84    |
|    explained_variance | -0.0681  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -7.47    |
|    reward             | 0.700627 |
|    std                | 1        |
|    value_loss         | 5.39     |
------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 370        |
|    iterations         | 200        |
|    time_elapsed       | 2          |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -2.86      |
|    explained_variance | 0          |
|    learning_rate      | 0.0007     |
|    n_updates    

## Trade

In [18]:

trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [19]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)


Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1023939.32
total_reward: 23939.32
total_cost: 68.68
total_trades: 824
Sharpe: 0.400
hit end!


In [20]:
df_actions.to_csv("action.csv", index=False) 
df_actions

,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600050.SH,600104.SH,600196.SH,600276.SH,600309.SH,600519.SH,600547.SH,600570.SH
date,,,,,,,,,,,,,,,
2019-08-01,1000,1000,0,1000,0,1000,0,0,1000,1000,0,0,0,1000,1000
2019-08-02,1000,1000,0,1000,0,1000,0,0,1000,1000,0,0,0,1000,1000
2019-08-05,1000,1000,0,1000,0,1000,0,0,1000,1000,0,0,0,1000,1000
2019-08-06,600,0,0,1000,0,1000,0,0,1000,1000,0,0,0,1000,1000
2019-08-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Backtest

### matplotlib inline

In [25]:
df_account_value.columns=["time","account_value"]
print(df_account_value)

plotter = ReturnPlotter(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot()

           time  account_value
0    2019-08-01      1000000.0
1    2019-08-02      1000000.0
2    2019-08-05      1000000.0
3    2019-08-06      1000000.0
4    2019-08-07      1000000.0
..          ...            ...
99   2019-12-26      1000000.0
100  2019-12-27      1000000.0
101  2019-12-30      1000000.0
102  2019-12-31      1000000.0
103  2020-01-02      1000000.0

[104 rows x 2 columns]


In [ ]:
# ticket: SSE 50：000016
plotter.plot("000016")

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


IndexError: list index out of range

### CSI 300

In [ ]:
baseline_df = plotter.get_baseline("399300")

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

==============DRL Strategy Stats===========
perf_stats_all: Annual return         -0.025429
Cumulative returns    -0.010473
Annual volatility      0.327358
Sharpe ratio           0.085284
Calmar ratio          -0.154956
Stability              0.328101
Max drawdown          -0.164107
Omega ratio            1.016573
Sortino ratio          0.114543
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.040802
Daily value at risk   -0.041132
Alpha                       NaN
Beta                        NaN
dtype: float64


In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")

==============Baseline Strategy Stats===========
perf_stats_all: Annual return         NaN
Cumulative returns    NaN
Annual volatility     NaN
Sharpe ratio          NaN
Calmar ratio          NaN
Stability             NaN
Max drawdown          NaN
Omega ratio           NaN
Sortino ratio         NaN
Skew                  NaN
Kurtosis              NaN
Tail ratio            NaN
Daily value at risk   NaN
Alpha                 NaN
Beta                  NaN
dtype: float64
